# Assignment 3 ( Question 4) Know the hatred!

## Instructions
- Run this notebook on ```Google Colab(preferable)```
- Write your code and analysis in the indicated cells.
- Ensure that this notebook runs without errors when the cells are run in sequence.
- Do not attempt to change the contents of other cells. 

## Packages Used
- all the packages that are imported in the template code

## Submission
- Rename the notebook to `<roll_number>_Assignment3_Q4.ipynb`.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import re
import sklearn
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
!pip install transformers
import transformers
from transformers import AutoTokenizer, AutoModel

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 77 kB 7.4 MB/s 
     |████████████████████████████████| 6.5 MB 38.3 MB/s 
     |████████████████████████████████| 895 kB 44.6 MB/s 
     |████████████████████████████████| 596 kB 52.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


#Dataset

In the training data, the comments are labelled as one or more of the six categories; toxic, severe toxic, obscene, threat, insult and identity hate. This is essentially a multi-label classification problem.

The dataset here is a multi-label classification dataset. To understand multi-label classification datasets, you can refer here: https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/ 

.


Here, we will be using Binary relevance to solve our multi-label classification problem



**Binary Relevance**: This is probably the simplest which treats each label as a separate single classification problems. The key assumption here though, is that there are no correlation among the various labels.

Finally, we will be summing up the accuracies obtained of all labels to obtain the final accuracy


In [ ]:
# loading the dataset

train_df = pd.read_csv('q4_data/train.csv')
test_df = pd.read_csv('q4_data/test.csv')
label_df = pd.read_csv('q4_data/test_labels.csv')
#dataset understanding
train_df.sample(5)
test_df = pd.concat([test_df,label_df],axis =1 , join = 'inner')

In [ ]:
test_df.head(5)

id                                       comment_text  \
0  00001cee341fdb12  Yo bitch Ja Rule is more succesful then you'll...   
1  0000247867823ef7  == From RfC == \n\n The title is fine as it is...   
2  00013b17ad220c46  " \n\n == Sources == \n\n * Zawe Ashton on Lap...   
3  00017563c3f7919a  :If you have a look back at the source, the in...   
4  00017695ad8997eb          I don't anonymously edit articles at all.   

                 id  toxic  severe_toxic  obscene  threat  insult  \
0  00001cee341fdb12     -1            -1       -1      -1      -1   
1  0000247867823ef7     -1            -1       -1      -1      -1   
2  00013b17ad220c46     -1            -1       -1      -1      -1   
3  00017563c3f7919a     -1            -1       -1      -1      -1   
4  00017695ad8997eb     -1            -1       -1      -1      -1   

   identity_hate  
0             -1  
1             -1  
2             -1  
3             -1  
4             -1

In [ ]:
# let's see the total rows in train, test data and the numbers for the various categories
print('Total rows in test is {}'.format(len(test_df)))
print('Total rows in train is {}'.format(len(train_df)))
# print(train_df[cols_target].sum())


Total rows in test is 153164
Total rows in train is 159571


In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text


In [ ]:
train_df['comment_text'] = train_df['comment_text'].map(lambda com : clean_text(com))
test_df['comment_text'] = test_df['comment_text'].map(lambda com : clean_text(com))

In [ ]:
# train_df = train_df.drop('char_length',axis=1)
X = train_df.comment_text
test_X = test_df.comment_text
print(X.shape, test_X.shape)


(159571,) (153164,)


Now, we will be experimenting this classification problem with different classifiers. We will be using pre-trained models inorder to obtain the required embeddings. Here, we will be using BERT and Universal Sentence Encoder embeddings.


Understand how Elmo and USE(Universal Sentence Encoder) works here. You can use any of both to train your classifiers.

To understand how different sentence embeddings work, you can refer to this link: https://neptune.ai/blog/how-to-code-bert-using-pytorch-tutorial 

Please give it a look before you start working with transfer learning/ sentence embeddings like BERT, Elmo, Ulm-fit, USE.


In [ ]:
#loading universal sentence encoder

import tensorflow as tf
import tensorflow_hub as hub
tf.compat.v1.enable_eager_execution()
# Load pre-trained universal sentence encoder model
use_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
# elmo_embed = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

In [ ]:
def get_use_embedding(sentence):
  embedding = use_embed([sentence])
  embedding = embedding.numpy()
  return embedding[0]

In [ ]:
#to obtain bert embeddings ( you need not completely understand how it works)
#you can directly use the function get_bert_embedding to obtain the embedding


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = model.to(device)

def get_bert_embedding(sentence):
  encoded_input = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt', max_length = 512)
  encoded_input = encoded_input.to(device)
  with torch.no_grad():
    model_output = model(**encoded_input)
  sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
  return sentence_embedding[0]


In [ ]:
#Obtain X_test and X_train 
train_df_bert = np.array(train_df['comment_text'].map(lambda com : get_bert_embedding(com).to('cpu').numpy()))
test_df_bert = np.array(test_df['comment_text'].map(lambda com : get_bert_embedding(com).to('cpu').numpy()))


In [ ]:
# np.save('train_df_bert', train_df_bert)
# np.save('test_df_bert', train_df_bert)

In [ ]:
train_df_use = np.array(train_df['comment_text'].map(lambda com : get_use_embedding(com)))
test_df_use = np.array(test_df['comment_text'].map(lambda com : get_use_embedding(com)))

In [ ]:
# # np.save('train_df_use', train_df_use)
# np.save('test_df_use', train_df_use)

In [ ]:
# df = pd.DataFrame([1,2,3,4])
# z = np.array(df.apply(lambda x:x**2))

In [ ]:
# # type(z)

# train_df_use = np.load('/content/drive/MyDrive/q4_data/train_df_use.npy',allow_pickle=True)
# test_df_use = np.load('/content/drive/MyDrive/q4_data/test_df_use.npy', allow_pickle=True)
# train_df_bert = np.load('/content/drive/MyDrive/q4_data/train_df_bert.npy', allow_pickle=True)
# test_df_bert = np.load('/content/drive/MyDrive/q4_data/test_df_bert.npy', allow_pickle=True)


In [ ]:
# train_df_use.shape

(159571, 512)

In [ ]:
# train_df_bert.shape

(159571, 768)

In [ ]:
# test_df_use.shape

(153164, 512)

In [ ]:
# test_df_bert.shape

(153164, 768)

In [ ]:
rows = train_df[ (train_df['toxic'] == -1) | (train_df['severe_toxic'] == -1)  |(train_df['obscene'] == -1) | (train_df['threat'] == -1) | (train_df['insult'] == -1) | (train_df['identity_hate'] == -1)].index
train_df.drop(rows,inplace = True)


In [ ]:
train_df_bert = np.delete(train_df_bert , rows,axis = 0)
train_df_use = np.delete(train_df_use , rows,axis = 0)

In [ ]:
rows = test_df[ (test_df['toxic'] == -1) | (test_df['severe_toxic'] == -1) | (test_df['obscene'] == -1) | (test_df['threat'] == -1) | (test_df['insult'] == -1) | (test_df['identity_hate'] == -1)].index
test_df.drop(rows,inplace = True)


In [ ]:
test_df_bert = np.delete(test_df_bert , rows,axis = 0)
test_df_use = np.delete(test_df_use , rows,axis = 0)


In [ ]:
# train_df_use.shape

(159571, 512)

In [ ]:
# train_df_bert.shape


(159571, 768)

In [ ]:
# test_df_use.shape


(63978, 512)

In [ ]:
# test_df_bert.shape


(63978, 768)

In [ ]:
col = ['toxic'	,'severe_toxic',	'obscene',	'threat',	'insult',	'identity_hate']
y_test = test_df[col]
y_train = train_df[col]

# Classifiers
Here, we use different classifiers for our classification problem. You can directly import classifiers for first two.


*   Naive Bayes classifier
*   Support Vector Machine
*   Multi-layer Perceptron(MLP)

In [ ]:
!pip install scikit-multilearn

In [ ]:
#calculate the accuracies obtained, u can use sklearn metrics
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.problem_transform import BinaryRelevance

In [ ]:
# Naive Bayes classifier 
# from sklearn.naive_bayes import MultinomialNB

from sklearn.naive_bayes import GaussianNB


nbclassifier = BinaryRelevance(GaussianNB())
nbclassifier.fit(train_df_use,y_train)
predictions = nbclassifier.predict(test_df_use)
print('Accuracy: ',(accuracy_score(y_test, predictions)))

Accuracy:  0.7983838194379318


In [ ]:
# Naive Bayes classifier 
# from sklearn.naive_bayes import MultinomialNB

from sklearn.naive_bayes import GaussianNB


nbclassifier = BinaryRelevance(GaussianNB())
nbclassifier.fit(train_df_bert,y_train)
predictions = nbclassifier.predict(test_df_bert)
print('Accuracy for bert nb: ',(accuracy_score(y_test, predictions)))

Accuracy for bert nb:  0.7384413392103536


In [ ]:
#Support Vector Machines
# from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import LinearSVC

classifier = BinaryRelevance(LinearSVC())

classifier.fit(train_df_use,y_train)

predictions = classifier.predict(test_df_use)
print('Accuracy: ',(accuracy_score(y_test, predictions)))

Accuracy:  0.8930257275938603


In [ ]:
#Support Vector Machines
# from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import LinearSVC

classifier = BinaryRelevance(LinearSVC())

classifier.fit(train_df_bert,y_train)

predictions = classifier.predict(test_df_bert)
print('Accuracy for bert svm: ',(accuracy_score(y_test, predictions)))

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

Accuracy for bert svm:  0.8910406702303917


In [ ]:
print('Accuracy for bert svm: ',(accuracy_score(y_test, predictions)))

Accuracy for bert svm:  0.8910406702303917


Which classifier of the three performed better, and why do you think so ?

Ans:
SVM performed better than other 2 models in terms of accuracy. The main assumption in Naive Bayes is taht all the features presenta are independent of each other. Whereas SVM tries to find some relation between features.


The SVM can be used and works very well in Multilabel classification with techniques like One vs rest and One vs One. Where as the MLP works very good foe sinle class classification. In case of sklearn MLP we were not able to apply different activation in different layers, the performance would have increased. This helped the SVM to get slightly better accuracy than sklearn MLP.

#Multi-layer Perceptron

For Multi-layer Perceptron, first you will be implementing your own articial Neural network, and then compare it with sklearn's MLP layer.


Implement your own artificial Neural Network (MLP)
Few Steps to be followed(for your reference):

*   Initialise the weights randomly
*   Decide upon the number of layers you wanna have and number of neurons associated in each layer. ( Have a clear idea of what will be the size of the weight matrices).
*   Write the code for forward and backward propogations
*   Decide upon the loss function
*   Train the model
*   Predict the labels after training the model




In [ ]:
import math
import random

In [ ]:
def relu(input):
  ans = []
  for i in input:
    if i > 0:
      ans.append(i)
    else:
      ans.append(0)
  return np.array(ans)

In [ ]:
def relu_deri(input):
  ans = []
  for i in input:
    if i > 0:
      ans.append(1)
    else:
      ans.append(0)
  return np.array(ans)

In [ ]:
def sigmoid(input):
  ans = [ 1/(1 + math.exp(-i)) for i in input]
  return np.array(ans)

In [ ]:
def sigmoid_deri(input):
  l1 = sigmoid(input)
  ans = [i*(1-i) for i in l1]
  return np.array(ans)

In [ ]:
def error_calc(out_,output):
  return np.mean(np.power(output-out_, 2))

In [ ]:
def grad_error_calc(out_,output):
  return (out_ -output )

In [ ]:

class Layer:
  def __init__(self,input,output):
    weights = []
    for i in range(output):
      temp = [np.random.normal(loc=0.0, scale = np.sqrt(2/(input+output))) for i in range(input)]
      weights.append(np.array(temp))
    self.weights = np.array(weights).transpose()
    self.biases = np.array([random.uniform(0,1) for i in range(output)])
  

  def foward(self, input1):
    # input1 = deepcopy(input1)
    self.input = input1
    # w = weights.transpose()
    return (input1 @ self.weights) + self.biases

  def backward(self,error,lr):
    
    # print(self.input.transpose().shape)
    # print(error.shape)
    
    grad_weights = np.dot(self.input.reshape((self.input.shape[0],1)), error.reshape((error.shape[0],1)).transpose())
    # print(grad_weights.shape)
    grad_biases = error.mean(axis=0)*self.input.shape[0] 
    self.weights = self.weights - lr * grad_weights
    self.biases = self.biases - lr * grad_biases
    grad_out = np.dot(error, self.weights.transpose())
    return grad_out
 

In [ ]:
class Activation:
  def __init__(self,acti,deriv):
    self.acti = acti
    self.acti_deri = deriv
  
  def foward(self,input):
    self.input = input
    return self.acti(input)
  
  def backward(self,error,lr):
    # print(self.acti_deri(self.input).shape)
    # print(np.dot(error,self.acti_deri(self.input)).shape)
    return np.multiply(error,self.acti_deri(self.input))

In [ ]:
class Sequential:
  def __init__(self, learning_rate):
    self.layers = []
    self.lr = learning_rate
    self.accuracy = dict()
  
  def add(self , layer):
    self.layers.append(layer)
  
  def foward_pass(self,input):
    out = input
    for l in self.layers:
      out = l.foward(out)
    return out

  # def backward_pass(self,input):
  #   pass
  def train(self,input , output,epochs =2 ):
    for epoch in range(epochs):
      print('Epoch: ',epoch+1)
      for i in range(input.shape[0]):
        out_ = self.foward_pass(input[i])
        # error = error_calc(out_,output[i])
        p =np.array([1 if o>=0.5 else 0 for o in out_])
        self.accuracy[i] = (accuracy_score(output[i], p))
        grad_error = grad_error_calc(out_,output[i])
        n = len(self.layers)
        # print(error)
        for l in range(len(self.layers)):
          # print('layer',n-1-l) 
          grad_error = self.layers[n-1-l].backward(grad_error,self.lr)
    return self.accuracy.values()
  def test(self,input,output):
    # acu = 0
    # wr = 0
    pred = []
    accu = []
    for i in range(input.shape[0]):
      out_ = self.foward_pass(input[i])
      for i in range(out_.shape[0]):
        if(out_[i]>=0.5):
          out_[i]=1
        else:
          out_[i]=0
      pred.append(out_)
      accu.append(accuracy_score(output[i], out_))
    pred= np.array(pred)
    return (pred,accu)
    # print('Accuracy ',(accuracy_score(output, pred)))
    
    

In [ ]:
# you can remove returns statements if u think they are not required
# you can also add functions if needed

# def initialise_weights():
#   return
  

# def loss_function():
#   return 


# def forward_layer():
#   return 


# def backward_propogation():
#   return 




In [ ]:
# def train():
  # pass

In [ ]:
# from pandas.io import parsers
# #report accuracies
# #here, you need to predict for each of the label as we have trained classifiers in such a way

# def predict():
#   parsers



In [ ]:
#report accuracies on the test dataset 
model = Sequential(0.01)
model.add(Layer(512,16))
model.add(Activation(relu,relu_deri))
model.add(Layer(16,32))
model.add(Activation(relu,relu_deri))
model.add(Layer(32,16))
model.add(Activation(relu,relu_deri))
model.add(Layer(16,6))
model.add(Activation(sigmoid,sigmoid_deri))



In [ ]:
model.train(train_df_use,y_train.values,3)

Epoch:  1
Epoch:  2
Epoch:  3


dict_values([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.3333333333333333, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8333333333333334, 1.0, 1.0, 1.0, 0.8333333333333334, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.3333333333333333, 0.5, 0.8333333333333334, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6666666666666666, 1.0, 1.0, 1.0, 0.3333333333333333, 0.5, 1.0, 0.6666666666666666, 0.8333333333333334, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6666666666666666, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6666666666666666, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.3333333333333333, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8333333333333334, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.

In [ ]:
pred , accu = model.test(test_df_use , y_test.values)

In [ ]:
# pred[0]

In [ ]:
print('Accuracy ',(accuracy_score(y_test.values, pred)))

Accuracy  0.90241958173122


In [ ]:
print(accu)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [ ]:
#report accuracies on the test dataset 
model_bert = Sequential(0.01)
model_bert.add(Layer(768,16))
model_bert.add(Activation(relu,relu_deri))
model_bert.add(Layer(16,32))
model_bert.add(Activation(relu,relu_deri))
model_bert.add(Layer(32,16))
model_bert.add(Activation(relu,relu_deri))
model_bert.add(Layer(16,6))
model_bert.add(Activation(sigmoid,sigmoid_deri))



In [ ]:
model_bert.train(train_df_bert,y_train.values,3)

Epoch:  1
Epoch:  2
Epoch:  3


dict_values([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.3333333333333333, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8333333333333334, 1.0, 1.0, 1.0, 0.8333333333333334, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.3333333333333333, 0.5, 0.8333333333333334, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6666666666666666, 1.0, 1.0, 1.0, 0.3333333333333333, 0.5, 1.0, 0.6666666666666666, 0.8333333333333334, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6666666666666666, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6666666666666666, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.3333333333333333, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8333333333333334, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.

In [ ]:
pred,accu = model_bert.test(test_df_bert , y_test.values)

In [ ]:
print('Accuracy ',(accuracy_score(y_test.values, pred)))

Accuracy  0.90241958173122


In [ ]:
print(accu)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

Now, compare your implemented MLP with the sklearn MLP layer.

In [ ]:
#report accuracies both on train dataset and test dataset

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

sk_mlp = MLPClassifier(hidden_layer_sizes=(16,32,16),activation = 'logistic',solver = 'adam')
sk_mlp.fit(train_df_use,y_train)
predictions = sk_mlp.predict(test_df_use)
print('Accuracy ',(accuracy_score(y_test, predictions)))

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Accuracy  0.8881646816093032


In [ ]:
#report accuracies both on train dataset and test dataset

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

sk_mlp = MLPClassifier(hidden_layer_sizes=(16,32,16),activation = 'logistic',solver = 'adam')
sk_mlp.fit(train_df_bert,y_train)
predictions = sk_mlp.predict(test_df_bert)
print('Accuracy bert mlp ',(accuracy_score(y_test, predictions)))

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Accuracy bert mlp  0.8692831910969395


Which MLP performed better? 
Understand and Analyse.

The MLP built by me gave better accuracy. The reason can be that I ahve used different activation function in different layers. In Sklearn MLP the activation funvtion remained same in all layers
Because our MLP is just a vanilla MLP, sklearn's MLP would have done better in an some environment. Sklearn MLP would have worked better in the data where same activation in all layers would have been suffice.
So, while SkLearn's MLP performed better, our prediction measurement was poor.